In [2]:
pip install geopy

     |████████████████████████████████| 112kB 28.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests

from geopy.geocoders import Nominatim
from pandas.io.json  import json_normalize 
from sklearn.cluster import KMeans

In [9]:
neighs_and_coords = pd.read_csv('list_of_postal_codes_and_coordinates.csv')
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
            len(neighs_and_coords['Borough'].unique()),
            neighs_and_coords.shape[0]
    )
)
neighs_and_coords.shape

The dataframe has 11 boroughs and 103 neighborhoods.


(103, 5)

In [11]:
address    = 'Toronto, CA'
geolocator = Nominatim(user_agent="luiz_alberto_capstone_project")
location  = geolocator.geocode(address)
latitude  = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto city are {} {}.'.format(latitude, longitude))

The geographical coordinate of Toronto city are 43.653963 -79.387207.


In [12]:
map_of_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
neighs_and_coords = neighs_and_coords[neighs_and_coords['Borough'].str.contains('Toronto') == True]
for lat, lng, borough, neighborhood in zip(neighs_and_coords['Latitude'], neighs_and_coords['Longitude'], neighs_and_coords['Borough'], neighs_and_coords['Neighborhood']):
    label = '{} {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186c',
        fill_opacity=0.7
    ).add_to(map_of_toronto)
map_of_toronto

In [13]:
CLIENT_ID = 'ZQCXQRLF431VWBPSYBLBEA24U0VO3Q5UCAAXCGR2FW1QV4WT' # your Foursquare ID
CLIENT_SECRET = 'VKNIKUXSARSLMUNZB2EBTAXVQR0KJJWHBVI3N2Z5MEBCXFZT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZQCXQRLF431VWBPSYBLBEA24U0VO3Q5UCAAXCGR2FW1QV4WT
CLIENT_SECRET:VKNIKUXSARSLMUNZB2EBTAXVQR0KJJWHBVI3N2Z5MEBCXFZT


In [14]:
neighs_and_coords.loc[37, 'Neighborhood']
neighborhood_latitude = neighs_and_coords.loc[37, 'Latitude'] 
neighborhood_longitude = neighs_and_coords.loc[37, 'Longitude'] 

neighborhood_name = neighs_and_coords.loc[37, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [15]:
LIMIT  = 100
radius = 500
url    = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZQCXQRLF431VWBPSYBLBEA24U0VO3Q5UCAAXCGR2FW1QV4WT&client_secret=VKNIKUXSARSLMUNZB2EBTAXVQR0KJJWHBVI3N2Z5MEBCXFZT&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e3c43e16d8c56001b55b65a'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Domino's Pizza,Pizza Place,43.679058,-79.297382
4,Upper Beaches,Neighborhood,43.680563,-79.292869


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat,  
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
neighs_and_coords_venues = getNearbyVenues(names=neighs_and_coords['Neighborhood'],
                                   latitudes=neighs_and_coords['Latitude'],
                                   longitudes=neighs_and_coords['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [22]:
print(neighs_and_coords_venues.shape)
neighs_and_coords_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [23]:
neighs_and_coords_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16


In [24]:
print('There are {} uniques categories.'.format(len(neighs_and_coords_venues['Venue Category'].unique())))


There are 230 uniques categories.


In [25]:
neighs_and_coords_onehot = pd.get_dummies(neighs_and_coords_venues[['Venue Category']], prefix="", prefix_sep="")
neighs_and_coords_onehot['Neighborhood'] = neighs_and_coords_venues['Neighborhood'] 
fixed_columns = [neighs_and_coords_onehot.columns[-1]] + list(neighs_and_coords_onehot.columns[:-1])
neighs_and_coords_onehot = neighs_and_coords_onehot[fixed_columns]
neighs_and_coords_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
neighs_and_coords_onehot.shape

(1706, 230)

In [27]:
neighs_and_coords_grouped = neighs_and_coords_onehot.groupby('Neighborhood').mean().reset_index()
neighs_and_coords_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.02,...,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.01
1,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.018519,0.0,0.0,0.00,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.043478,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.00


In [28]:
neighs_and_coords_grouped.shape

(39, 230)

In [29]:
num_top_venues = 5

for hood in neighs_and_coords_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighs_and_coords_grouped[neighs_and_coords_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2    Steakhouse  0.04
3          Café  0.04
4      Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop  0.09
1       Nightclub  0.09
2            Café  0.09
3  Breakfast Spot  0.09
4     Yoga Studio  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Garden Center  0.06
3          Comic Shop  0.06
4         Pizza Place  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3             P

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighs_and_coords_venues_sorted = pd.DataFrame(columns=columns)
neighs_and_coords_venues_sorted['Neighborhood'] = neighs_and_coords_grouped['Neighborhood']

for ind in np.arange(neighs_and_coords_grouped.shape[0]):
    neighs_and_coords_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighs_and_coords_grouped.iloc[ind, :], num_top_venues)

neighs_and_coords_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Bar,Café,Asian Restaurant,Cosmetics Shop,Restaurant,Bakery,Burger Joint,Breakfast Spot
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Seafood Restaurant,Farmers Market,Cheese Shop,Steakhouse,Café,Greek Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Nightclub,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Butcher
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar


In [33]:
kclusters = 5
neighs_and_coords_grouped_clustering = neighs_and_coords_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighs_and_coords_grouped_clustering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [34]:
neighs_and_coords_merged = neighs_and_coords
neighs_and_coords_merged['Cluster Labels'] = kmeans.labels_
neighs_and_coords_merged = neighs_and_coords_merged.join(neighs_and_coords_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
neighs_and_coords_merged

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pizza Place,Pub,Trail,Health Food Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Liquor Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Pizza Place,Park,Fish & Chips Shop,Pub,Fast Food Restaurant,Sushi Restaurant,Ice Cream Shop,Liquor Store,Burrito Place,Burger Joint
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Sandwich Place,Cheese Shop,Pet Store
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Lawyer,Bus Line,Swim School,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Department Store,Hotel,Park,Breakfast Spot,Sandwich Place,Food & Drink Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Spa,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Cosmetics Shop
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Gym,Greek Restaurant,Japanese Restaurant
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Playground,Summer Camp,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Pub,Coffee Shop,Sushi Restaurant,Supermarket,Sports Bar,Liquor Store,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant


In [35]:
#MAP
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(neighs_and_coords_merged['Latitude'], neighs_and_coords_merged['Longitude'], neighs_and_coords_merged['Neighborhood'], neighs_and_coords_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 0, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Pizza Place,Pub,Trail,Health Food Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Liquor Store
42,East Toronto,0,Pizza Place,Park,Fish & Chips Shop,Pub,Fast Food Restaurant,Sushi Restaurant,Ice Cream Shop,Liquor Store,Burrito Place,Burger Joint
43,East Toronto,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Sandwich Place,Cheese Shop,Pet Store
44,Central Toronto,0,Park,Lawyer,Bus Line,Swim School,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
45,Central Toronto,0,Department Store,Hotel,Park,Breakfast Spot,Sandwich Place,Food & Drink Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
46,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Spa,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Cosmetics Shop
47,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Gym,Greek Restaurant,Japanese Restaurant
48,Central Toronto,0,Playground,Summer Camp,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
49,Central Toronto,0,Pub,Coffee Shop,Sushi Restaurant,Supermarket,Sports Bar,Liquor Store,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant


In [37]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 1, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Burger Joint,Flower Shop,Pub,BBQ Joint,Indian Restaurant,Middle Eastern Restaurant,History Museum


In [38]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 2, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Downtown Toronto,2,Coffee Shop,Ice Cream Shop,Italian Restaurant,Juice Bar,Burger Joint,Café,Japanese Restaurant,Sandwich Place,Bakery,Salad Place
69,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Cocktail Bar,Restaurant,Japanese Restaurant,Beer Bar,Hotel,Seafood Restaurant,Park


In [39]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 3, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,Restaurant,Burger Joint,Bar,Japanese Restaurant,Deli / Bodega,Hotel,Asian Restaurant


In [40]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 4, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4,Garden,Home Service,Ice Cream Shop,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
